<a href="https://colab.research.google.com/github/SergioSarmientoRosales/Training-Regression_Models/blob/main/Training_Regression_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import time
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
import matplotlib.pyplot as plt

# Load data
file_path = "/content/model_psnr_result.csv"  # Replace with the actual file path
data = pd.read_csv(file_path, header=None)
X = np.array([eval(row[0]) for row in data.values])
y = data.iloc[:, -1]
# Defining models and their hyperparameters for tuning
models = [
    ('Linear Regression', LinearRegression(), {'fit_intercept': [True, False]}),
    ('Ridge Regression', Ridge(), {'alpha': [0.1, 1, 10]}),
    ('Lasso Regression', Lasso(), {'alpha': [0.1, 1, 10]}),
    ('ElasticNet Regression', ElasticNet(), {'alpha': [0.1, 1, 10], 'l1_ratio': [0.2, 0.5, 0.8]}),
    ('Decision Tree Regression', DecisionTreeRegressor(), {'max_depth': [None, 5, 10, 15, 20], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4, 6]}),
    ('Random Forest Regression', RandomForestRegressor(), {'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}),
    ('XGBoost Regression', GradientBoostingRegressor(), {'n_estimators': [100, 200, 300], 'max_depth': [3, 4, 5], 'learning_rate': [0.01, 0.1, 0.2], 'subsample': [0.8, 0.9, 1.0]}),
    ('AdaBoost Regression', AdaBoostRegressor(), {'n_estimators': [50, 100, 200], 'learning_rate': [0.5, 1.0, 1.5]}),
    ('Extra Trees Regression', ExtraTreesRegressor(), {'n_estimators': [100, 200, 300], 'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]}),
    ('KNN Regression', KNeighborsRegressor(), {'n_neighbors': [1, 3, 5, 7, 9, 11], 'weights': ['uniform', 'distance'], 'p': [1, 2, 3, 4], 'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']}),
    ('Support Vector Regression', SVR(), {'C': [0.1, 1, 10], 'gamma': ['scale', 'auto'], 'kernel': ['linear', 'rbf', 'poly']})
]

# Preparing to store the results
results = []

# Using K-Fold for cross-validation with 10 splits
kf = KFold(n_splits=5)

# Iterating over each model to train and evaluate them
for model_name, model, param_grid in models:
    print(f"Working on {model_name}...")
    start_time = time.time()

    # Conducting grid search for the best hyperparameters
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=kf, scoring='r2')
    grid_search.fit(X, y)
    best_model_instance = grid_search.best_estimator_

    # 10-fold cross-validation on the best configuration
    cv_scores_mse = cross_val_score(best_model_instance, X, y, cv=kf, scoring='neg_mean_squared_error')
    avg_cv_score_mse = np.abs(np.mean(cv_scores_mse))

    # Additional evaluation with R^2
    cv_scores_r2 = cross_val_score(best_model_instance, X, y, cv=kf, scoring='r2')
    avg_cv_score_r2 = np.abs(np.mean(cv_scores_r2))

    duration = time.time() - start_time

    # Storing the results for later visualization
    results.append((model_name, duration, avg_cv_score_mse, avg_cv_score_r2))

    # Outputting the progress
    print(f"{model_name} - Time: {duration:.2f} sec, Avg MSE: {avg_cv_score_mse:.4f}, Avg R^2: {avg_cv_score_r2:.4f}")

# Visualizing the performance of each model
labels, times, mses, r2s = zip(*results)

# Creating a figure for bar plots
plt.figure(figsize=(18, 6))

# Plot for Training Time
plt.subplot(1, 3, 1)
plt.barh(labels, times, color='salmon')
plt.xlabel('Training Time (seconds)')
plt.title('Model Training Time Comparison')

# Plot for Mean Squared Error
plt.subplot(1, 3, 2)
plt.barh(labels, mses, color='lightgreen')
plt.xlabel('Mean Squared Error')
plt.title('Model Performance Comparison (MSE)')

# Plot for R^2
#plt.subplot(1, 3, 3)
#plt.barh(labels, r2s, color='skyblue')
#plt.xlabel('R^2')
#plt.title('Model Performance Comparison (R^2)')

# Adjusting layout for better readability
plt.tight_layout()
plt.savefig('model_performance_comparison.png')
plt.show()

Working on Linear Regression...
Linear Regression - Time: 0.94 sec, Avg MSE: 18.5383, Avg R^2: 0.4053
Working on Ridge Regression...
Ridge Regression - Time: 0.50 sec, Avg MSE: 18.5229, Avg R^2: 0.4034
Working on Lasso Regression...
Lasso Regression - Time: 0.34 sec, Avg MSE: 16.9799, Avg R^2: 0.2130
Working on ElasticNet Regression...
ElasticNet Regression - Time: 0.94 sec, Avg MSE: 16.9799, Avg R^2: 0.2130
Working on Decision Tree Regression...
Decision Tree Regression - Time: 6.12 sec, Avg MSE: 21.1536, Avg R^2: 0.7481
Working on Random Forest Regression...


KeyboardInterrupt: 